<a href="https://colab.research.google.com/github/KorStats/ocr_receipt/blob/main/%EA%B3%BC%EC%A0%95/(%EB%85%B8%EC%9D%B4)_%EC%9B%90%EB%B3%B8S%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%A4%91%EB%B6%84%EB%A5%98_%ED%95%99%EC%8A%B5%EA%B2%B0%EA%B3%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 구글드라이브에서 파일 읽어오기

In [1]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

#깃허브에서 KoBERT 파일 로드
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
#GPU 사용
device = torch.device("cuda:0")
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

     |████████████████████████████████| 47.3 MB 79.7 MB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 5.1 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595734 sha256=340d513bfdafa54145d46dbf1e63fea4c8b6d56892233c657c96a57e29cd8d4b
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 5.0 MB/s 
     |████████████████████████████████| 769 kB 5.0 MB/s 
     |████████████████████████████████| 895 kB 46.0 MB/s 
     |████████████████████████████████| 3.0 MB 28.4 MB/s 
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-7sns1bp5
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/p

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
 기본   산업분류자동화	'Colab Notebooks'  'My Drive'


In [34]:
import pandas as pd
df=pd.read_csv('/mydrive/산업분류자동화/1. 실습용자료.txt', sep = "|", encoding = "euc-kr")
ts=pd.read_csv('/mydrive/산업분류자동화/답안 작성용 파일.csv', encoding = "euc-kr")

In [35]:
df=df[df['digit_1']=='S']
ts=ts[ts['digit_1']=='S']

In [36]:
df=df.reset_index(drop=True)

## 데이터 전처리

In [37]:
df.head(10)

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal
0,id_0000001,S,95,952,카센터에서,자동차부분정비,타이어오일교환
1,id_0000017,S,95,952,사업장에서,고객요청으로,자동차세차업
2,id_0000029,S,96,961,미용실에서,일반인대상,컷.파마
3,id_0000034,S,96,969,세탁소에서,고객의 의뢰를 받아,가정용 세탁물 세탁
4,id_0000037,S,94,949,교회,기독교계통종교활동,종교서비스
5,id_0000040,S,94,949,교회에서,기독교계통의 종교활동,종교서비스
6,id_0000058,S,94,949,교회,기독교계통종교활동,종교서비스
7,id_0000069,S,96,961,미용실에서,고객을 대상으로,"염색,퍼머,커트"
8,id_0000074,S,96,961,미용실에서,고객을 대상,"컷트,파마,염색서비스"
9,id_0000077,S,96,961,이발소에서,고객을대상으로,컷트 면도 등 두발이용서비스


In [38]:
df['text']=df['text_obj'].map(str)+" "+df['text_mthd'].map(str)+" "+df['text_deal'].map(str)
i=[]
for sentence in df['text']:
    a=sentence.replace('nan', '')
    i.append(a)
df['text']=pd.DataFrame(i)
df=df.drop(['text_obj','text_mthd', 'text_deal'], axis=1) # 기존에 있던 세개의 text변수 삭제
df

,AI_id,digit_1,digit_2,digit_3,text
0,id_0000001,S,95,952,카센터에서 자동차부분정비 타이어오일교환
1,id_0000017,S,95,952,사업장에서 고객요청으로 자동차세차업
2,id_0000029,S,96,961,미용실에서 일반인대상 컷.파마
3,id_0000034,S,96,969,세탁소에서 고객의 의뢰를 받아 가정용 세탁물 세탁
4,id_0000037,S,94,949,교회 기독교계통종교활동 종교서비스
...,...,...,...,...,...
100391,id_0999945,S,95,952,사업장에서 고객요청으로 자동차경정비
100392,id_0999958,S,94,941,지부에서 회원들의 친목도모 회원관리
100393,id_0999967,S,96,961,미용실 일반인대상 미용
100394,id_0999979,S,95,951,매장에서 고객의 의뢰에의해 핸드폰수리


In [39]:
# 결측값 확인
df.isnull().sum()

AI_id      0
digit_1    0
digit_2    0
digit_3    0
text       0
dtype: int64

In [40]:
df.shape

(100396, 5)

In [41]:
df['digit_2'].value_counts()

96    56210
94    23625
95    20561
Name: digit_2, dtype: int64

In [44]:
df['digit_3'].value_counts().sort_index()

941     1612
942      433
949    21580
951     1508
952    12142
953     6911
961    43040
969    13170
Name: digit_3, dtype: int64



#### **추후 과제: 분류가 현재 매우 불균형함 / 정규화를 이용해서 균형을 맞추면 정확도가 올라가는지 확인 필요**


In [46]:
df.loc[(df['digit_3'] == 941), 'digit_3'] = 0  
df.loc[(df['digit_3'] == 942), 'digit_3'] = 1  
df.loc[(df['digit_3'] == 949), 'digit_3'] = 2  
df.loc[(df['digit_3'] == 951), 'digit_3'] = 3  
df.loc[(df['digit_3'] == 952), 'digit_3'] = 4
df.loc[(df['digit_3'] == 953), 'digit_3'] = 5
df.loc[(df['digit_3'] == 961), 'digit_3'] = 6
df.loc[(df['digit_3'] == 969), 'digit_3'] = 7

In [47]:
data_list = []
for q, label in zip(df['text'], df['digit_3'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

print(data_list[:5])

[['카센터에서 자동차부분정비 타이어오일교환', '4'], ['사업장에서 고객요청으로 자동차세차업', '4'], ['미용실에서 일반인대상 컷.파마', '6'], ['세탁소에서 고객의 의뢰를 받아 가정용 세탁물 세탁', '7'], ['교회 기독교계통종교활동 종교서비스', '2']]


## Train data & test data

In [48]:
from sklearn.model_selection import train_test_split             
dataset_train, dataset_test = train_test_split(data_list, test_size=0.1, random_state=0) #추후에 test_size 수정 필요

# KoBERT 입력 데이터로 만들기

In [49]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [50]:
# Setting parameters
max_len = 64
batch_size = 64 #32,16,8로 감소시키기
warmup_ratio = 0.1
num_epochs = 5 #추후 num_epochs 증가할 필요 있음
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

 ### 추후 과제 : 쓰이는 parameters 최적화 필요

In [51]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [52]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [53]:
data_train[0]

(array([   2, 2609, 7184, 3807, 5440, 5816, 7078,  517, 6596, 7753, 6461,
        6023, 6305,    3,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(14, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 7)

In [54]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


# KoBERT 학습 모델 만들기

In [55]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=8,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [56]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


# KoBERT 모델 학습시키기

In [57]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/1412 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 2.171976089477539 train acc 0.140625
epoch 1 batch id 201 loss 0.8144997954368591 train acc 0.5076181592039801
epoch 1 batch id 401 loss 0.25847452878952026 train acc 0.6955657730673317
epoch 1 batch id 601 loss 0.2978101074695587 train acc 0.774022462562396
epoch 1 batch id 801 loss 0.02668772265315056 train acc 0.8201857053682896
epoch 1 batch id 1001 loss 0.256478875875473 train acc 0.8494474275724275
epoch 1 batch id 1201 loss 0.029888980090618134 train acc 0.8699391132389676
epoch 1 batch id 1401 loss 0.048647888004779816 train acc 0.884870182012848
epoch 1 train acc 0.8856096099368055


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

epoch 4 test acc 0.9893511146496815


  0%|          | 0/1412 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.003166681155562401 train acc 1.0
epoch 5 batch id 201 loss 0.07607090473175049 train acc 0.9941697761194029
epoch 5 batch id 401 loss 0.008629722520709038 train acc 0.9931031795511222
epoch 5 batch id 601 loss 0.06690617650747299 train acc 0.9936824043261231
epoch 5 batch id 801 loss 0.0012586392695084214 train acc 0.9938943508114857
epoch 5 batch id 1001 loss 0.00610865605995059 train acc 0.9939123376623377
epoch 5 batch id 1201 loss 0.002979167038574815 train acc 0.9940934637801832
epoch 5 batch id 1401 loss 0.006075793877243996 train acc 0.994245182012848
epoch 5 train acc 0.9942789483002833


  0%|          | 0/157 [00:00<?, ?it/s]

epoch 5 test acc 0.9888535031847133


In [59]:
#학습 모델 저장
torch.save(model, 'S_ori_so_model.pt')